#0. Carga de datos

In [ ]:
# Importamos las librerías habituales
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pylab as plt
import numpy as np



---


##Cómo levantar el df limpio

In [ ]:
directory = '/content/drive/My Drive/DatosTP/limp'
df = pd.read_pickle('/content/drive/My Drive/DatosTP/limp1.pickle')

In [ ]:
for i in range(2,14):
  path = directory + str(i) + '.pickle'
  df1 = pd.read_pickle(path)
  df = df.append(df1)

df
dfRank = df

In [ ]:
df["raices"] = df["raicesl"]
df["text"] = df["dream"]

#Junta y arma un solo "texto" con las raíces
df['raices_unidas'] = df['raices'].apply(" ".join)

Es casi igual que antes, solo que con 13 archivos limp#.pickle  
También cambiaron las columnas:  
['cohesion', 'intent', 'lucidity', 'raicesl', 'rating', 'technique',
       'url', 'user', 'dream', 'additional_comments', 'themes', 'settings',
       'characters', 'emotions', 'activities']

---





---

##Cómo levantar el df Viejo


In [ ]:
#Correr solo esto para tener un df de 9k entradas
#directory = '/content/drive/My Drive/Dreams/data'
#df = pd.read_pickle('/content/drive/My Drive/Dreams/data1.pickle')

In [ ]:
#Todos los pickles (ESTÁ PUESTO HASTA EL 5: TODOS ES HASTA EL 15)
#for i in range(2,5):
#  path = directory + str(i) + '.pickle'
#  df1 = pd.read_pickle(path)
#  df = df.append(df1, ignore_index=True)

df

#1. Limpieza previa

In [ ]:
#Hay algunos repetidos...
print(df["raices"][35427])
print(df["raices"][35428])
print(df["url"][35427])

In [ ]:
df

# 2. ¿Qué sueña la gente?

#Análisis de sentimientos

In [ ]:
import nltk

#no sé si usamos todo esto
nltk.download([
     "names",
     "stopwords",
     "state_union",
     "twitter_samples",
     "movie_reviews",
     "averaged_perceptron_tagger",
     "vader_lexicon",
     "punkt",
])

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

df["sentiment"] = df["text"].apply(sia.polarity_scores)   #TARDA BASTANTE

In [ ]:
df["sentiment"]

In [ ]:
df["neg"] = pd.Series
df["pos"] = pd.Series

for i in df["sentiment"].index:
  df["neg"][i] = df["sentiment"][i]["neg"]
  df["pos"][i] = df["sentiment"][i]["pos"]


In [ ]:
#Los que tienen más de 0,5 en positivo
df[df["pos"]>0.5]

In [ ]:
#Más de 0,7 en negativo
df[df["neg"]>0.7]

In [ ]:
#Histograma negativo
df["neg"].hist()

# Word2vec

In [ ]:
from gensim.models.phrases import Phrases, Phraser

input = [row.split() for row in df['raices_unidas']] # separamos en una lista

phrases = Phrases(input, min_count=30, progress_per=1000)

bigram = Phraser(phrases)

sentences = bigram[input] # obtenemos las palabras junto con bigramas

In [ ]:
import multiprocessing

from gensim.models import Word2Vec

cores = multiprocessing.cpu_count()

w2v_model = Word2Vec(min_count=20, # ignora palabras cuya frecuencia es menor a esta
                     window=2, # tamanio de la ventana de contexto
                     size=300, # dimension del embedding
                     sample=6e-5, # umbral para downsamplear palabras muy frecuentes
                     alpha=0.03, # tasa de aprendizaje inicial (entrenamiento de la red neuronal)
                     min_alpha=0.0007, # tasa de aprendizaje minima
                     negative=20, # penalidad de palabras muy frecuentes o poco informaitvas
                     workers=cores) # numero de cores para entrenar el modelo

w2v_model.build_vocab(sentences, progress_per=10000) # construye el vocabulario

### ENTRENA EL MODELO
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
### PRECOMPUTA DISTANCIAS (mas rapido)
w2v_model.init_sims(replace=True)


KeyboardInterrupt: ignored

In [ ]:
w2v_model.wv.most_similar(positive=["sex"])

# Tópicos

In [ ]:
# Importamos las librerías habituales
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pylab as plt
import numpy as np

In [ ]:
# Objetos de sklearn para hacer tópicos
from sklearn.feature_extraction.text import CountVectorizer # Contador de frecuencia
from sklearn.feature_extraction.text import TfidfTransformer # Creador de tf-idf

# Algoritmos de descomposición de tópicos
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:

# Creamos el objeto contador de palabras, pidiéndole que remueve
# las stopwords, los términos que aparecen en un único documento (min_df)
# y los términos que aparecen en más del 70% de los documentos (max_df).
# Esto es para eliminar palabras raras (o errores de tipeo) y
# términos que seguramente son stopwords no incluídos en la lista
count = CountVectorizer(min_df = 2, max_df = 0.70, stop_words = stopwords)

# Ajustamos con los datos. Acá especificamente creamos una matriz documentos-términos
x_count = count.fit_transform(df['raices_unidas'])

# Dimensions de la matriz doc-tér
print(x_count.shape)

In [ ]:
# Creamos el objeto tf-idf. Le decimos además que devuelva los
# vectores documento con norma euclídea igual a 1 (norm = 'l2')
tfidf = TfidfTransformer(norm = 'l2')

# Creamos la matriz tf-idf a partir de la matriz de frecuencias
x_tfidf = tfidf.fit_transform(x_count)

In [ ]:
# Elijamos la cantidad de tópicos
n_components = 5

# Construímos el objeto NMF con los tópicos indicados
nmf = NMF(n_components = n_components)

# Aplicamos sobre nuestros datos
x_nmf = nmf.fit_transform(x_tfidf)

# Dimensión de la matriz transformada
print(x_nmf.shape)

In [ ]:
# Objeto índice: término de nuestro vocabulario
vocabulary = {item: key for key, item in count.vocabulary_.items()}

# Para cada componente
for n in range(n_components):

  # Ordenamos una lista del largo de nuestro vocabulario según el peso en cada componente y nos quedamos con los primeros 10
  list_sorted = sorted(range(nmf.components_.shape[1]), reverse = True, key = lambda x: nmf.components_[n][x])[:10]

  # Printeamos los términos asociados a los valores más grande de cada una de las componentes
  print(', '.join([vocabulary[i] for i in list_sorted]))
  print('\n')

In [ ]:
# WordClouds
wc_atributos = {'height' : 800,
                'width' : 1200,
                'background_color' : 'white',
                'max_words' : 20
                } # Defino los parámetros que les voy a pasar a los wordclouds

# Creo la figura
fig, axs = plt.subplots(n_components, figsize = (6,20))

# Recorro para todas las componentes
for n in range(n_components):

  # 10 términos más pesados
  list_sorted = sorted(range(len(vocabulary)), reverse = True, key = lambda x: nmf.components_[n][x])[:10]

  # Diccionario término: peso
  comp_dict = {vocabulary[i]: nmf.components_[n][i] for i in list_sorted}

  # Creo el wordlcoud
  wc = WordCloud(**wc_atributos # De esta forma, le estoy diciendo a la función que expanda el diccionario de atributos de forma tal de que entienda lo que quiero que haga
                 ).generate_from_frequencies(comp_dict)

  axs[n].set_title('Tópico {}'.format(n))
  axs[n].imshow(wc)
  axs[n].axis('off')

plt.show()

In [ ]:
# Normalizador
from sklearn.preprocessing import Normalizer

# Creamos un objeto para normalizar a que la suma dé 1
norm = Normalizer('l1')

# Sobreescribimos sobre la matriz de documentos-tópicos
x_nmf = norm.fit_transform(x_nmf)

# Guardemos en el dataframe esta información
for n in range(n_components):
  df['nmf_comp{}'.format(n)] = x_nmf[:,n]

df.head()

In [ ]:
df_metodo = df.groupby('technique').mean()

# Inspeccionemoslo
df_metodo

In [ ]:
# El eje x es la década
x = df_metodo.index

# El eje y son las distribuciones
y = df_metodo[['nmf_comp{}'.format(i) for i in range(n_components)]].to_numpy()

plt.figure(figsize = (8,5))
plt.stackplot(x, y.T) # Stackplot: sirve para graficar distribuciones
#plt.xlim([0, 90])
plt.ylim([0, 1.00])
plt.yticks([])
plt.xlabel('Método')
plt.legend(['Tópico {}'.format(i) for i in range(n_components)], loc = (1.05, 0.60))
plt.show()

# 3. ¿Cómo sueña?

In [ ]:
#Nos quedamos solo con los que contestaron green o red
index_red = df["intent"]=="red"
index_green = df["intent"]=="green"

index_or = index_green + index_red
df = df[index_or]

#Tabla de contingencia de lucidity e intent
pd.crosstab(df["intent"], df["lucidity"], margins=True)

In [ ]:
#Qué técnicas usaron
df["technique"].value_counts()

In [ ]:
#Tabla de contingencia de técnica e intent
pd.crosstab(df["intent"], df["technique"], margins = True)

In [ ]:
#Tabla de contingencia de lucidity y técnica
pd.crosstab(df["lucidity"], df["technique"], margins = True)

#Nubes de palabras por intention: green/red

In [ ]:
#Me quedo con las raíces
raices = df["raices_unidas"].transpose()
raices_green = df["raices_unidas"][df["intent"]=="green"].transpose()
raices_red = df["raices_unidas"][df["intent"]=="red"].transpose()

#Las junto todos en un solo texto de raices
todos_textos_green = " ".join(raices_green)
todos_textos_red = " ".join(raices_red)

#Las cuento
print("Hay en total:", raices.size, "relatos")  #hay vacíos?
print("De los cuales son red:", raices_red.size)
print("La cant de green son:", raices_green.size)


In [ ]:

# Cargamos del paquete nltk las stopwords a la lista "stopwords"
import nltk
nltk.download('stopwords') # hay que descargar este modulo en particular

stopwords = nltk.corpus.stopwords.words('english')

print(stopwords)

In [ ]:
#Limpieza adicional para este set
stopwords = stopwords + ["quot", "nbsp", "additional", "comment", "rsquo", "rdquo", "unfamiliar character"] #add emotions?

In [ ]:
print(df["raices"][151])

In [ ]:
from wordcloud import WordCloud # importo la funcion WordCloud
import matplotlib.pyplot as plt

# Creo el objeto WordCloud sacando la lista de stopwords
wc = WordCloud(stopwords=stopwords, background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

plt.rcParams['figure.figsize'] = [16,12] # tamaño de los plots

#Genero green
wc.generate(todos_textos_green)
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Green sin stopwords")
plt.show()

#Genero red
wc.generate(todos_textos_red)
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Red sin stopwords")
plt.show()


In [ ]:
# Ahora con stopwords
wc = WordCloud(background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

wc.generate(todos_textos_green)  # acá le pido que genere los WC a partir del texto de cada año
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Green con stopwords")
plt.show()

wc.generate(todos_textos_red)  # acá le pido que genere los WC a partir del texto de cada año
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Red con stopwords")
plt.show()

In [ ]:
#Armo listas de raices green/red con y sin stopwords:
#Primero con stopwords:
lista_raices_green = df["raices"][df["intent"]=="green"].transpose()
lista_raices_red = df["raices"][df["intent"]=="red"].transpose()

#Función que remueve de una lista de strings los elementos en stopwords
def sw_remover3(lista):
   return [value for value in lista if not value in stopwords]

#Aplico la función y lo guardo en una nueva columna
lista_raices_green_sinsw = lista_raices_green.apply(sw_remover3)
lista_raices_red_sinsw = lista_raices_red.apply(sw_remover3)

# Se borraron los stopwords, por ejemplo:
print(lista_raices_red[4])
print(lista_raices_red_sinsw[4])

Sobre el dataset:  
-1 significa que no había datos al scrapear  
No hay Nan entre los elementos.  
Hay ruido

In [ ]:
# Importamos las librerías habituales
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pylab as plt
import numpy as np

In [ ]:
# Objetos de sklearn para hacer tópicos
from sklearn.feature_extraction.text import CountVectorizer # Contador de frecuencia
from sklearn.feature_extraction.text import TfidfTransformer # Creador de tf-idf

# Algoritmos de descomposición de tópicos
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation

#RANKEO


In [ ]:

ratings = dfRank["rating"].value_counts().to_dict()

print("Total de sueños no rankeados = " + str(ratings[-1]))
print("Total de sueños rankeados = " + str(ratings[5]+ratings[4]+ratings[3]+ratings[2]+ratings[1]))
print("Total de sueños rankeados positivos = " + str(ratings[5]+ratings[4]))
print("Total de sueños rankeados negativos = " + str(ratings[2]+ratings[1]))
print("Total de sueños rankeados neutros = "+str(ratings[3]))

Creo los dataframes segun rankeo positivo o negativo

In [ ]:
dfRank = dfRank.loc[dfRank["rating"]>0] #elimino los suenios no rankeados == -1
dfPosRank = dfRank.loc[dfRank["rating"]>3] #asumo rank pos == mayor a 3
dfNegRank = dfRank.loc[dfRank["rating"]<3] #asumo rank neg == menor a 3

rank = dfRank["user"].value_counts().to_dict()
rankPos = dfPosRank["user"].value_counts().to_dict()
rankNeg = dfNegRank["user"].value_counts().to_dict()


##Son los suenios rankeados altos de los mismos usuarios?
SI

In [ ]:
data = list(rankPos.values())

plt.pie(data, labels= rankPos.keys())
plt.show()

## quiero ver cuantos usuarios escribieron que porcentaje del total
 es decir, se que aproximadamente 30 usuarios escribieron el 50% del total de los rankeados positivos, 200 usuarios el 5% y 8000 usuarios el otro 45%


 no se que tipo de grafico usar para mostrar que

In [ ]:
cantUsuarios = np.zeros(5)
cantPublicaciones = np.zeros(5)
etiquetas = ["1-10","11-30","31-50","51-100","101-453"]


keys = list(rankPos.keys())
analizar5porciento = []
analizar95porciento = []


for i in range(len(rankPos.keys())):
  key = keys[i]
  if(rankPos[key] < 11):
    cantPublicaciones[0] += rankPos[key]
    cantUsuarios[0] += 1
    #para analisis del otro parte
    analizar95porciento.append(key)
  elif(rankPos[key] < 31):
    cantPublicaciones[1] += rankPos[key]
    cantUsuarios[1] += 1
    #para mi grafico d analisis ams adelante
    analizar5porciento.append(key)
  elif(rankPos[key] < 51):
    cantPublicaciones[2] += rankPos[key]
    cantUsuarios[2] += 1
    #para mi grafico d analisis ams adelante
    analizar5porciento.append(key)
  elif(rankPos[key] < 101):
    cantPublicaciones[3] += rankPos[key]
    cantUsuarios[3] += 1
    #para mi grafico d analisis ams adelante
    analizar5porciento.append(key)
  else:
    cantPublicaciones[4] += rankPos[key]
    cantUsuarios[4] += 1
    #para mi grafico d analisis ams adelante
    analizar5porciento.append(key)


for i in range(len(cantUsuarios)):
  cantUsuarios[i] = str(cantUsuarios[i])


print(cantPublicaciones,cantUsuarios)


fig, axs = plt.subplots(nrows = 1, ncols = 2)
axs[0].set_title("Suenios")
axs[0].pie(cantPublicaciones, labels = cantUsuarios ,colors = plt.get_cmap('Set3').colors, autopct='%1.2f%%')
#axs[1].set_title["Usuarios"]
axs[1].pie(cantUsuarios, labels = etiquetas ,colors = plt.get_cmap('Set3').colors, autopct='%1.2f%%')
plt.show()

grafico 1: porcentaje de entradas y usuarios que las escribieron/// suma de publicaciones que hicieron los usuarios entre rango

grafico 2: usuarios sobre entradas que publicaron
esto me lleva a querer analizar a este 5% de los usuarios///cantidad de  usuarios con x entradas entre

In [ ]:
plt.bar(height = cantPublicaciones, x= etiquetas)
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
plt.hist(rankPos.values(), bins = 453,color = "darkcyan")
plt.xticks(np.arange(0,453,step=20))
plt.ylabel("Usuarios")
plt.xlabel("Cantidad de entradas publicadas")
plt.yscale('log')
plt.show()

In [ ]:
lista_de_entradas = [x for x in rankPos.values() if x < 50]

plt.figure(figsize=(15,15))
plt.hist(lista_de_entradas, bins = 50,color = "darkcyan")
plt.xticks(np.arange(0,50,step=1))
plt.ylabel("Usuarios")
plt.xlabel("Cantidad de entradas publicadas")
plt.yscale('log')
plt.show()

In [ ]:
lista_de_entradas3 = [x for x in rankNeg.values() if x < 453]

plt.figure(figsize=(15,15))
plt.hist(lista_de_entradas3, bins = 453,color = "m")
plt.xticks(np.arange(0,453,step=20))
plt.ylabel("Usuarios")
plt.xlabel("Cantidad de entradas publicadas")
plt.yscale('log')
plt.show()

In [ ]:
lista_de_entradas2 = [x for x in rankNeg.values() if x < 50]

plt.figure(figsize=(15,15))
plt.hist(lista_de_entradas2, bins = 50,color = "m")
plt.xticks(np.arange(0,50,step=1))
plt.ylabel("Usuarios")
plt.xlabel("Cantidad de entradas publicadas")
plt.yscale('log')
plt.show()

##quiero analizar al 5% de los usuarios que publicaron mas del 50% de los posRank

voy a hacer un promedio de todas sus publicaciones y ver si tienen mas publicaciones positivas que negativas

In [ ]:

promediar = np.zeros(2)
for n in range(len(analizar5porciento)):
  i = analizar5porciento[n]
  if(i in rankPos):
    promediar[0] += rankPos[i]
  if(i in rankNeg):
    promediar[1] += rankNeg[i]

data = promediar / len(analizar5porciento)

promediar1 = np.zeros(2)
for n in range(len(analizar95porciento)):
  i = analizar95porciento[n]
  if(i in rankPos):
    promediar1[0] += rankPos[i]
  if(i in rankNeg):
    promediar1[1] += rankNeg[i]

data1 = promediar1 / len(analizar95porciento)



plt.figure(figsize=(4,8))
plt.bar(height=promediar , x=["Positivo","Negativo"], color = ["blue","red"], width=0.8)

plt.show()
print(promediar)

In [ ]:
promediar1 = np.zeros(2)
for n in range(len(analizar95porciento)):
  i = analizar95porciento[n]
  if(i in rankPos):
    promediar1[0] += rankPos[i]
  if(i in rankNeg):
    promediar1[1] += rankNeg[i]

data1 = promediar1 / len(analizar95porciento)

plt.figure(figsize=(4,8))
plt.bar(height=promediar1 , x=["Positivo","Negativo"], color = ["blue","red"])
plt.show()
print(promediar1)

yo de aca concluyo que tienen muchos buenos rankeados pero porque escriben muchos-> esto me lleva a descartar una hipotesis de mentiras/hacerlo para obtener puntaje alto



vemos que las sumas de los suenios negativos y los positivos nos dice que el 5% que tienen muchos bien rankeados es porque escriben mucho. mientras que el 95% esta mas parejo en cuanto al rank?? nose si meter esto /// creo q no

['cohesion', 'intent', 'lucidity', 'raicesl', 'rating', 'technique', 'url', 'user', 'dream', 'additional_comments', 'themes', 'settings', 'characters', 'emotions', 'activities']




Quiero hacer porcentajes por cada usuario q publico alguna vez algun suenio para ver como suelen ser rankeados sus suenios

In [ ]:

dicPorcentajes = dict()
rankIT = list(rank.keys())
for i in range(len(rank.keys())):
  key = rankIT[i]
  if(key in rankPos):
    pos = (rankPos[key]*100)/rank[key]
    pos = round(pos,1)
  else:
    pos = 0
  if(key in rankNeg):
    neg = (rankNeg[key]*100)/rank[key]
    neg = round(neg,1)
  else:
    neg = 0
  neu = 100 -(pos+neg)
  dicPorcentajes[key] = [pos,neg,neu]


#busqueda
busquedaPositivos = []
it = list(dicPorcentajes.keys())
for i in range(len(dicPorcentajes.keys())):
  key = it[i]
  #if(dicPorcentajes[key][0]>dicPorcentajes[key][1] and rank[key]>8):
  if(dicPorcentajes[key][0]>dicPorcentajes[key][1] and rank[key]>8):
    busquedaPositivos.append([key,dicPorcentajes[key][0],dicPorcentajes[key][1],rank[key]])

#busquedaPositivos#[user,%pos,%neg,cantPublicaciones]

de hecho existen usuarios con todos sus suenios rankeados positivos pero estos no superan los 10 suenios publicados

---nose q mas hacer con esto

